# 1. Import Modules and Set Global Variables

In [ ]:
"""
This script is made for Grad.AI for S.E.A safety competition.

- Before start, please make sure you have put testing features, labels under `testfeatures`,
  and `testlabels` folder respectively. Besides, make sure only testing file(s) (.csv) are under 
  `testfeatures` folder and only one label file (.csv) is under `testlabels` folder. 

- `isTesting` = False is used for local cross validation. 
   It shoulbe be set to True for testing purpose.

- `fun_switches` is used to switch on/off respective feature egineering function.
   By setting some of the switches off, this step can be made faster at a sacrifice of some predicting power.

- `fun_name`, `fun_list`, ... `num_features_fun` are global variables. Please don't change them. 
""" 

import math
import os
import time
import pywt
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import auc, roc_curve

from scipy.signal import savgol_filter
from scipy.fftpack import fft
from peakdetect import peakdet

%matplotlib inline
warnings.filterwarnings('ignore')



########## Set isTesting = True for testing #################

isTesting = True


####### Set all Switches and ensemble option to True ########
# It takes about 30 mins to run on a 400MB testing dataset 
# with all switches below turned on and the ensemble option enabled 
# 
# However, the finalised model takes about half of the time.

ensemble = True
fun_switches = [True, True, True, True, True, True] 


############ Global Variables. Don't Modify ##################

fun_names = ['idf','fftf','cwt','stat','xsec','pkf']
num_features_fun = [13, 15*10, 20*10, 15*10, 20*10, (5*3)*10] 

# 2. Concatenate Feature Files and Generate Helper Files

In [ ]:
def init_processing(isTesting = isTesting):
    """
    Concatenate and sort data
    """
    bgn = time.time()
    
    if isTesting:
        folder, lbl_folder = 'testfeatures', 'testlabels'
    else:
        folder, lbl_folder = 'features', 'labels'
    
    
    #read label file
    yfiles = os.listdir(os.getcwd() + '/' + lbl_folder) 
    yfiles = [file for file in yfiles if file.split('.')[-1] == 'csv']
    if len(yfiles) != 1:
        raise Exception('Make sure {} folder has and only has .csv file as label'.format(lbl_folder))
    yfile = yfiles[0]
    labels = pd.read_csv(lbl_folder+'/'+yfile)
    labels = labels[~labels['bookingID'].duplicated(keep = 'first')]
    
    
    # read feature file(s)
    files = os.listdir(os.getcwd() + '/' + folder) 
    files.sort()
    cls = ['bookingID', 'Accuracy', 'Bearing', 
       'acceleration_x','acceleration_y', 'acceleration_z', 
       'gyro_x', 'gyro_y', 'gyro_z',
       'second', 'Speed']
    data  = pd.DataFrame(columns = cls)    
    for f in files:
        if f.split('.')[-1] == 'csv' and f != 'new.csv' and f != 'reference.csv' :
            concat_bgn = time.time()
            data = pd.concat([data, pd.read_csv(folder + '/' + f)])
            concat_end = time.time()
            print('concat',f, ' ',concat_end - concat_bgn)            
    data = data.sort_values(by = ['bookingID', 'second'])
    data = data.reset_index(drop = True)
    data = data.merge(labels, how = 'left', on = 'bookingID')    
    data.to_csv(folder+'/'+'new.csv', index = False)
    print('Merging and sorting', str(time.time() - concat_end ))
     
        
    # generate tb: a table of individual trip information  
    tb_bgn = time.time()
    groups = data.groupby(['bookingID'])
    heads = groups.head(1).index
    tails = groups.tail(1).index
    ids = groups.head(1)['bookingID'].values.astype(int)
    lbls = groups.head(1)['label'].values.astype(int)
    tb = np.vstack((ids,heads,tails,lbls)).T 
    np.save(lbl_folder+'/'+'lbls.npy', lbls)
    np.save(lbl_folder+'/'+'tb.npy', tb)
    print('generate trip information table', time.time() - tb_bgn)
    
    
    # generate reference statistics 
    if isTesting:
        ref_data = pd.read_csv('features/reference.csv', index_col = 0)
    else:
        ref_bgn = time.time()
        ref_cls = cls
        ref_cls = [col for col in ref_cls if col != 'bookingID' and col != 'label']
        ref_data = pd.DataFrame(columns = ref_cls, index = ['max','min','1pct','99pct'])
        for col in ref_data.columns:
            ref_data.loc['max', col] = np.abs(data[col]).max()
            ref_data.loc['min', col] = np.abs(data[col]).min()
            ref_data.loc['1pct', col] = np.percentile(data[col], [1])[0]
            ref_data.loc['99pct', col] = np.percentile(data[col], [99])[0]  
        ref_data.to_csv('features/reference.csv')
        print('Generating reference statistics', time.time() - ref_bgn)
           
    print('Total time: ', time.time() - bgn)    
    return data, tb, ref_data

data, tb, ref_data = init_processing(isTesting)

# 3. Feature Engineering

In [ ]:
#1. bookingID features
def get_idf(trip):
    """
    Return ID features based on assumption that 
    'bookingID' is generated not fully by random
    """
    ID = trip['bookingID'].values[0]
    return [int(i) for i in list('0'*13+str(ID))[-13:]]


# 2. fast fourier transform (FFT) features
def get_fftf(trip,  n = 10): # n is the number of low frequency components
    """
    Frequency domain features
    """
    features = []
    for col in trip.columns:
        if col != 'bookingID' and col != 'label':
            fourier = fft(trip[col]) 
            fourier = np.abs(fourier)/trip.shape[0]
            interval = 2
            # skip DC component, keep n low freq components at a chosen interval
            if len(fourier) > n*interval+1:
                lowf = fourier[np.arange(1, n*interval+1,interval)]  
            else:
                length = math.floor(len(fourier)/interval)
                lowf = ([fourier[i*interval+1] for i in range(length)] + [0]*n)[:n]
            features += list(lowf) 
            features += list(np.percentile(fourier, [75, 90, 95, 97, 100])) #  distribution stats
    return features


# 3.  continuous wavelet transform (CWT) features
def get_cwt(trip, n = 20): # n is the number of scales
    """
    Return time-frequency coefficients with dimension reduced PCA
    """
    scales = np.arange(1, n + 1)
    features = []
    for col in trip.columns:
        if col != 'bookingID' and col != 'label':
            coef, freq = pywt.cwt(trip[col].values, scales,'morl')
            features += list(PCA(n_components = 1).fit_transform(coef).flatten())
    return features


# 4. statatistics features
def get_stat(trip, percentiles = [1, 3, 5, 10, 25, 75, 90, 95, 97, 99]):  
    """
    Return time domain statistics
    """
    features = []
    for col in trip.columns:
        if col != 'bookingID' and col != 'label':
            s = trip[col]
            basics = [s.mean(), s.mode().values[0], s.max(), s.min(), s.std()]            
            dist = np.percentile(trip[col], percentiles)
            features += basics
            features += list(dist)
    return features


# 5. cross-section features in ratio
def get_xsec(trip, n = 20): # n is the number of interval
    """
    Contextual driving style in ratios by comparing 
    driving style with peer statatistics
    """
    features = []
    for col in trip.columns:
        if col != 'bookingID' and col != 'label':
            ceiling = ref_data.loc['99pct',col]
            floor = ref_data.loc['1pct',col]
            interval = 1.*(ceiling - floor)/n
            parts = [floor+interval*i for i in range(n)]

            length = trip.shape[0]        
            cum_ratio_parts = [trip[trip[col] < part].shape[0]/length for part in parts]
            ratio_parts = [cum_ratio_parts[0]]
            for i in range(1,n):
                ratio_parts.append(cum_ratio_parts[i] - cum_ratio_parts[i-1])
            features += ratio_parts
    return features


# 6. peak groups features
def cal_peaks(maxtab, mintab):
    """
    Calculate STD, average intensity and frequency for given set of peaks found
    """
    if len(maxtab) + len(mintab) < 2:
        return [0,0,0]
    peaks = np.array([i for i in zip(maxtab[:, 1], mintab[:, 1])]).flatten()
    if len(maxtab) > len(mintab):
        peaks = np.hstack((peaks,  [maxtab[-1,1]]))
    duration = max((maxtab[-1,0] - maxtab[0,0]), mintab[-1,0] - maxtab[0,0])
    ave_freq = (len(maxtab) + len(mintab))/duration

    intensity = [np.abs(peaks[i+1]-peaks[i]) for i in range(len(peaks)-1)]
    ave_intensity = np.mean(intensity)
    std_intensity = np.std(intensity)
    return [ave_freq, ave_intensity, std_intensity]


def get_pkf(trip, window_size = 20, n = 5): # n is the number of interval
    """
    Statistics (STD, average intensity/frequency) of 
    local maxima/minima groups that found at different threholds
    """
    features = []
    for col in trip.columns:
        if col != 'bookingID' and col != 'label':
            high_ref = ref_data.loc['max', col]
            low_ref = ref_data.loc['min', col]
            interval = (high_ref - low_ref)/n
            series = trip[col]
            
            if len(trip[col]) > window_size + 1:
                series = savgol_filter(series, window_size + 1, 2)
                
            for i in range(n):
                maxtab, mintab = peakdet(series, max(low_ref + i * interval, interval))
                features += cal_peaks(maxtab, mintab)
    return features



def get_FeaturesnLabels(data, tb, isTesting = isTesting):
    """
    Common utility by different functions to calculate respective features
    """

    print('Getting_features ...')  
    report_interval = len(tb)//20
    bgn = time.time()
    
    features = []    
    for i in range(len(tb)):
        if i%report_interval == 0:
            print("progress",str(5*i//report_interval),"% ", str(time.time()-bgn))

        ID, head, tail, _ = map(int,tb[i])
        trip = data.iloc[head:tail,:]
        row = [] 
        
        for j in range(len(fun_list)):
            fun = fun_list[j]
            name = fun_names[j]
            num = num_features_fun[j]
            switch_on = fun_switches[j]            
            if switch_on:
                tripfeatures_byfun = fun(trip)
                actual_num = len(tripfeatures_byfun)
                if len(tripfeatures_byfun) != num:
                    raise Exception('Input num of features returned by '+name+
                                    ' is '+ str(actual_num)+' instead of '+ str(num))
                row += tripfeatures_byfun
            else:
                row += [0]*num
                
        features.append(row)
        
    features = np.array(features)
    
    file_name = 'features_{}.npy'.format(str(sum(num_features_fun)))
    if isTesting:
        np.save('testfeatures/' + file_name, features)
    else:
        np.save('features/' + file_name, features)
    print('Total time: ', time.time()-bgn)
    
    return features, tb[:,-1]

fun_list = [get_idf, get_fftf, get_cwt, get_stat, get_xsec, get_pkf]
features, labels = get_FeaturesnLabels(data, tb, isTesting = isTesting)

# 4. Ensemble

In [ ]:
def get_kfolds(n, n_folds = 5):   
    """
    Generate static splits for local cross validation
    """
    kf = StratifiedKFold(n_splits = n_folds, random_state = 3, shuffle = True)
    folds = list(kf.split(range(n), [0]*n))
    return folds


def get_estimator_preds(isTesting = isTesting):
    """
    Generate predictions from 6 base estimators based on features extracted
    """
    if isTesting:
        xfolder = 'testfeatures'
    else:
        xfolder = 'features'
        
    # sanity check    
    xfiles = os.listdir(os.getcwd() + '/' + xfolder)  
    xfile = [file for file in xfiles if file.split('.')[-1] == 'npy']
    if len(xfile) != 1 :
        raise Exception('''Make sure {} folder has and only has one .npy file'''.format(xfolder))
    xfile = xfile[0]
               
    estimators = [
        GaussianNB(),
        LogisticRegression(),
        MLPClassifier(),
        KNeighborsClassifier(5),
        DecisionTreeClassifier(),
        GradientBoostingClassifier(max_depth = 3)]
    
    features = []
    
    bgn = time.time() 
    for i in range(len(fun_names)):            
        if fun_switches[i]:
            round_bgn = time.time()    
            print('Generating predictions on', fun_names[i],'features ...')
            ed = np.cumsum(num_features_fun)[i]
            st = ed - num_features_fun[i]
            
            X = np.load('features/features_{}.npy'.format(str(sum(num_features_fun))))[:,st:ed]            
            y = np.load('labels/lbls.npy')
            
            folds = get_kfolds(X.shape[0], 5)            
            for estimator in estimators:
                idx = []
                preds4estimator = []
                for train_idx, test_idx in folds:
                    idx += list(test_idx)    
                    X_train, y_train = X[train_idx], y[train_idx]
                    if isTesting: 
                        X_test = np.load('testfeatures/'+xfile)[:,st:ed]
                    else:
                        X_test = X[test_idx]
                    partial_pred = estimator.fit(X_train, y_train).predict_proba(X_test)[:,0]
                    preds4estimator.append(partial_pred)
                if isTesting:
                    preds4estimator = np.mean(np.array(preds4estimator), axis = 0)
                else:
                    preds4estimator = np.array(preds4estimator).flatten()[np.argsort(idx)]
                features.append(preds4estimator)
            round_end = time.time()   
            print(round_end - round_bgn)
            
        else: # if switched off, simply write zeros
            num_trips = np.load(xfolder+'/'+xfile).shape[0]
            for _ in estimators:
                features.append([0]*num_trips)
                
    features = np.array(features).T
    np.save(xfolder+'/pred.npy', features)
    print('Total time: ', time.time() - bgn)    
    return features

column_indexing = []
pred_indexing = []
for i in range(len(fun_names)):
    column_indexing += [fun_switches[i]]*num_features_fun[i] 
    pred_indexing += [fun_switches[i]]*len(fun_list)
    
if ensemble:
    preds = get_estimator_preds(isTesting = isTesting)
else:
    pass

# 5. Display Model Performance 

In [ ]:
def ROC_display(isTesting = isTesting, fun_switches = fun_switches, ensemble = ensemble):
    """
    Display model performance using AUC as the metric
    """
    
    if isTesting: # testing the model on actual testing dataset
        num_f = sum(num_features_fun)
        if ensemble:
            X_train = np.load('features/pred.npy')[:,pred_indexing]
            X_test = np.load('testfeatures/pred.npy')[:,pred_indexing]
        else:
            X_train = np.load('features/features_{}.npy'.format(num_f))[:,column_indexing]
            X_test = np.load('testfeatures/features_{}.npy'.format(num_f))[:,column_indexing]
        y_train = np.load('labels/lbls.npy')
        y_test = labels 

        # For local sanity check on performance only. No need to uncomment
        all_idx, idx = np.load('labels/tb.npy')[:,0], np.load('testlabels/tb.npy')[:,0]
        selection = [True if ID not in idx else False for ID in all_idx]
        X_train = X_train[selection]
        y_train = y_train[selection]

    else:# local cross validation
        if ensemble:
            X = np.load('features/pred.npy')[:,pred_indexing]
        else:
            X = np.load('features/features_{}.npy'.format(sum(num_features_fun)))[:,column_indexing]
        y = np.load('labels/lbls.npy')

        kf = StratifiedKFold(n_splits = 5, random_state = 0, shuffle = True)
        folds = list(kf.split(X, y))
        train_index, test_index = folds[0]
        X_train, X_test    = X[train_index], X[test_index]
        y_train, y_test    = y[train_index], y[test_index]  
    y_test = y_test.astype(int)

    print('If isTesting:', isTesting)
    print('If ensemble:', ensemble)   
    print('Function names:', fun_names)
    print('If turned on:', fun_switches)
    print('Data shape:', X_train.shape, y_train.shape, X_test.shape, y_test.shape, '\n')

    colors = ['grey','black','purple','red']
    baseline_prob = [GaussianNB(),
                     KNeighborsClassifier(5),
                     LogisticRegression(),
                     GradientBoostingClassifier(max_depth = 3)]

    
    rocauc_results = {}
    for clf in baseline_prob:
        bgn = time.time() 
        clf_name = clf.__class__.__name__
        rocauc_results[clf_name] = {'auc':0,'fpr':[],'tpr':[]}

        clf.fit(X_train,y_train)
        rocauc_results[clf_name]['fpr'],rocauc_results [clf_name]['tpr'],_ = roc_curve(y_test, clf.predict_proba(X_test)[:, 1])
        rocauc_results[clf_name]['auc'] = auc(rocauc_results[clf_name]['fpr'], rocauc_results [clf_name]['tpr'])
        end = time.time()
        print(clf_name + " AUC score:",rocauc_results[clf_name]['auc'], " Time: " + str(end - bgn))

    plt.figure(figsize=(10,7))
    plt.plot([0, 1], [0, 1], color='navy',linestyle='--')

    for i,name in enumerate(rocauc_results.keys()):
        plt.plot(rocauc_results[name]['fpr'], rocauc_results[name]['tpr'], color=colors[i],label='{} (area = {:.2f})'.format(name,rocauc_results[name]['auc']))

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])

    plt.xlabel('False Positive Rate');
    plt.ylabel('True Positive Rate');
    plt.title('ROC Curve(Baseline)');
    plt.legend(loc="lower right");  
    return X_train, y_train, X_test, y_test

## 5.1 All feature generated included + Ensemble enabled 

In [ ]:
# with all switches turned on,
# it took around 30 minutes to model a 400MB testing dataset with MacBook Pro (2.6GHz)

X_train, y_train, X_test, y_test = ROC_display(isTesting = isTesting, fun_switches = fun_switches, ensemble = ensemble)
plt.savefig('ROC_allon')

## 5.2  All feature generated included + Ensemble disabled 

In [ ]:
# switching ensemble off saves 60% time with a majorly deteriorted AUC 

ensemble = False
fun_switches = [True, True, True, True, True, True] 

column_indexing = []
pred_indexing = []
for i in range(len(fun_names)):
    column_indexing += [fun_switches[i]]*num_features_fun[i] 
    pred_indexing += [fun_switches[i]]*len(fun_list) 
    
    
ROC_display(isTesting = isTesting, fun_switches = fun_switches, ensemble = ensemble)
plt.savefig('ROC_ensembledisabled')

## 5.3  The Best Trade-off between Cost and Performance
    
## BookingID and Cross-Sectional Features Included + Ensemble Enabled
     
          
                               
- One-layer ensemble model built on 12 predictions that made on 213 raw features 
- Took around 15 mins running time in total for a 400MB testing dataset on MacBook Pro (2.6GHz), inclusive of concatenating, sorting, preprocessing, engineering and ensembling

In [ ]:
# switching any function below off saves around 10% of time 
# (except for the first switch for bookingID features) with a minorly deteriorted AUC  
                      
ensemble = True
fun_switches = [True, False, False, False, True, False] 

column_indexing = []
pred_indexing = []
for i in range(len(fun_names)):
    column_indexing += [fun_switches[i]]*num_features_fun[i] 
    pred_indexing += [fun_switches[i]]*len(fun_list) 

    
ROC_display(isTesting = isTesting, fun_switches = fun_switches, ensemble = ensemble)
plt.savefig('ROC_onlyidNxsec')